In [1]:
import pandas as pd
ramen=pd.read_csv('Ramen_ratings_2020.csv')


In [2]:
reviews_2020=ramen.loc[ramen['URL'].str.contains('/2020/')]
reviews_2020

,ID,URL,Brand,Variety,Style,Country,Stars
0,3473,https://www.theramenrater.com/2020/04/05/3473-...,MyKuali,White Fish Broth Noodle,Pack,Malaysia,5
1,3472,https://www.theramenrater.com/2020/04/05/3472-...,MyKuali,Penang White Curry Noodle (New Recipe),Pack,Malaysia,5
2,3471,https://www.theramenrater.com/2020/04/05/3471-...,Ve Wong,Instant Oriental Noodles Soup Chinese Herb - G...,Pack,Taiwan,3.75
3,3470,https://www.theramenrater.com/2020/04/04/3470-...,Myojo,Ippeichan Rich & Sweet Yakisoba,Tray,Japan,5
4,3469,https://www.theramenrater.com/2020/04/03/3469-...,Mykuali,Penang Hokkien Prawn Rice Vermicelli Soup,Bowl,Malaysia,5
...,...,...,...,...,...,...,...
94,3379,https://www.theramenrater.com/2020/01/05/3379-...,Paldo,Mr. Kimchi Stirfried Kimchi Ramen,Pack,South Korea,5
95,3378,https://www.theramenrater.com/2020/01/04/3378-...,Yibin,Burning Noodle Hot & Spicy,Tray,China,0
96,3377,https://www.theramenrater.com/2020/01/03/3377-...,Sapporo Ichiban,Hokkaido Crab Nabe Shoyu Ramen,Cup,Japan,3.75
97,3376,https://www.theramenrater.com/2020/01/02/3376-...,Rooster,Chili Seafood Flavour Noodle Soup,Cup,Canada,3.25


In [3]:
import requests
from bs4 import BeautifulSoup as bs
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}

URLs = reviews_2020['URL']

def scrape(url):
    try:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        time.sleep(3)  # Add a 3-second delay between requests
        soup = bs(res.content, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print('There was a problem with the request:', e)
        return None

def get_element(content):
    if content:
        page = content.find('div', class_="entry-content")
        if page:
            all_p = page.find_all('p', text=True)
            return all_p
    return None

reviews = []
for url in URLs:
    review=[]
    html_content = scrape(url)
    if html_content:
        for p in get_element(html_content):
            review.append(p.text)
    reviews.append(review)


There was a problem with the request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [4]:
reviews[:5]

[['This is a brand new variety from Malsyai’s MyKuali. This company’s had three different varieties in the past end up on my global top ten rankings and after trying the vermicelli and cup versions, here we have the pack (you’ll see the other two in upcoming reviews. Let’s give it a try!',
  'The noodle block.',
  'A dry sachet of non-dairy creamer.',
  'A wet paste sachet.',
  'A clear dry garnish sachet.',
  '8-pack MyKuali/Malaysia Best Brand/Penang Prawn Noodle/Strong Prawn Flavor, Thick Hearty Broth Spicy Mouthfeel from Food Heaven, Penang Malaysia (105g/pack)',
  'Watch me cook on Instant Noodle Recipe Time!'],
 ['I gotta say, there have been a TON of new recipes for this instant noodle. I don’t know how many brands are changing recipes out there, but it’s nice to know when they do. This is something they do in Malaysia, specifically Penang, Malaysia, an awful lot. The folks at MyKuali told me how you might go to a hawker stall one day and have something really great and a week l

In [15]:
ramen_reviews= pd.DataFrame(reviews, columns=['reviews'])

In [17]:
ramen_reviews.to_csv('ramen_reviews.csv', index=False)